<a href="https://colab.research.google.com/github/JH-Y-324/first-repository/blob/master/sql_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1

🔥 힌트 2 — total_spent를 SELECT한 적이 없음

ORDER BY total_spent 하려면,
총 금액(total_spent)을 먼저 SELECT 또는 계산해야 합니다.

즉, SUM(price)를 계산하지 않은 상태에서
order by total_spent 를 쓸 수 없습니다.

🔥 힌트 3 — user_id로 묶어서 합계를 구해야 함

문제에서 요구한 핵심:

➤ 각 user_id가 총 얼마 썼는지를 집계

→ GROUP BY user_id

따라서 SUM(price)가 필요합니다.

In [ ]:
SELECT user_id,
       SUM(price) AS total_spent
FROM `project.dataset.ex1`
GROUP BY user_id
ORDER BY total_spent DESC
LIMIT 3;

#2

🔥 왜 서브쿼리가 필요한가?

WHERE절에서는 집계함수(AVG) 를 바로 쓸 수 없음

전체 평균을 먼저 계산해야 함

그 값과 현재 행의 total_bill을 비교해야 함

그래서:

내부 쿼리 → 평균 계산

외부 쿼리 → 평균보다 큰 행만 출력

In [ ]:
WHERE total_bill > (SELECT AVG(total_bill) FROM ...)

In [ ]:
SELECT *
FROM `firstproject-479902.firstdataset.ex2`
WHERE total_bill > (
    SELECT AVG(total_bill)
    FROM `firstproject-479902.firstdataset.ex2`
);

#3

In [ ]:
SELECT *
FROM `firstproject-479902.firstdataset.ex2`
WHERE total_bill > (
    SELECT AVG(total_bill)
    FROM `firstproject-479902.firstdataset.ex2`
)
ORDER BY table_id;   -- ← 여기

📌 SQL 실행 순서(중요)

FROM

WHERE

GROUP BY

HAVING

SELECT

ORDER BY

LIMIT

그래서 ORDER BY는 항상 마지막 부분에 둡니다.

#4

In [ ]:
SELECT PRODUCT_LINE,
       SUM(TOTAL_ORDER) AS total_sales
FROM `project.dataset.ex3`
GROUP BY PRODUCT_LINE
ORDER BY total_sales DESC
LIMIT 1;

#5

❗ “리뷰를 가장 많이 남긴 회원 1명을 먼저 찾고

그 회원의 모든 리뷰를 가져와서 정렬한다.”

즉, 서브쿼리가 필요합니다.

아래에서 정확한 구조를 설명드립니다.

✅ 핵심 단계 3가지
✔ 1단계: 리뷰를 가장 많이 남긴 회원(MEMBER_ID) 찾기

→ GROUP BY MEMBER_ID
→ ORDER BY COUNT(*) DESC LIMIT 1

In [ ]:
SELECT MEMBER_ID
FROM ex4_review
GROUP BY MEMBER_ID
ORDER BY COUNT(*) DESC
LIMIT 1

✔ 2단계: 그 MEMBER_ID의 리뷰 전체 가져오기

→ review 테이블을 member 테이블과 JOIN
→ WHERE MEMBER_ID = (1단계)

✔ 3단계: 정렬

REVIEW_DATE 오름차순

같은 날짜면 REVIEW_TEXT 오름차순

In [ ]:
SELECT m.MEMBER_NAME,
       r.REVIEW_TEXT,
       r.REVIEW_DATE
FROM `project.dataset.ex4_review` r
JOIN `project.dataset.ex4_member` m
      ON r.MEMBER_ID = m.MEMBER_ID
WHERE r.MEMBER_ID = (
    SELECT MEMBER_ID
    FROM `project.dataset.ex4_review`
    GROUP BY MEMBER_ID
    ORDER BY COUNT(*) DESC
    LIMIT 1
)
ORDER BY r.REVIEW_DATE ASC,
         r.REVIEW_TEXT ASC;

🟥 당신 쿼리에서 부족했던 점

“where review 테이블에서 count(member_id)가 가장 큰 사람”

이 문장은 개념은 맞지만,

COUNT를 WHERE에 바로 쓸 수 없음

서브쿼리 필요

JOIN 필요

최종 정렬 두 단계 필요

이 부분을 보완해야 전체 쿼리가 완성됩니다.